In [ ]:
import pandas as pd
imdb_url="https://raw.githubusercontent.com/tobypenk/data-bootcamp/main/week%2010/IMDB%20Dataset.csv"
imdb=pd.read_csv(imdb_url)
imdb

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
imdb.loc[1,"review"]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
imdb["review"] = imdb["review"].str.replace("<br />"," ").str.lower()
imdb["length"]= imdb["review"].apply(lambda x: len(x))
imdb.sort_values("length", ascending=False).loc[0,"review"]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.  the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.  it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.  i would say the main appeal of the show is due to the fact that it goes where other sh

In [ ]:
sentiment_dummies=pd.get_dummies(
    imdb["sentiment"],
    drop_first=True,
    prefix="sentiment"
)

imdb=pd.concat([imdb,sentiment_dummies],axis=1)
imdb

,review,sentiment,length,sentiment_positive
0,one of the other reviewers has mentioned that ...,positive,1731,1
1,a wonderful little production. the filming t...,positive,968,1
2,i thought this was a wonderful way to spend ti...,positive,906,1
3,basically there's a family where a little boy ...,negative,718,0
4,"petter mattei's ""love in the time of money"" is...",positive,1277,1
...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,988,1
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,622,0
49997,i am a catholic taught in parochial elementary...,negative,1260,0
49998,i'm going to have to disagree with the previou...,negative,1234,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X = imdb.drop("sentiment_positive",axis=1)
y = imdb["sentiment_positive"]
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
X_train

,review,sentiment,length
39087,that's what i kept asking myself during the ma...,negative,2298
30893,i did not watch the entire movie. i could not ...,negative,1594
45278,a touching love story reminiscent of in the m...,positive,502
16398,this latter-day fulci schlocker is a totally a...,negative,1061
13653,"first of all, i firmly believe that norwegian ...",negative,1700
...,...,...,...
11284,`shadow magic' recaptures the joy and amazemen...,positive,427
44732,i found this movie to be quite enjoyable and f...,positive,319
38158,avoid this one! it is a terrible movie. so wha...,negative,521
860,this production was quite a surprise for me. i...,positive,430


In [ ]:
vectorizer = TfidfVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train["review"])
X_test_vec = vectorizer.transform(X_test["review"])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lc = LogisticRegression()
lc.fit(X_train_vec, y_train )

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
from pandas.errors import AccessorRegistrationWarning
y_pred_lc=lc.predict(X_test_vec)
y_acc_lc=accuracy_score(y_pred_lc,y_test)
print("logistic regression accuracy was", y_acc_lc)

logistic regression accuracy was 0.8943


In [ ]:
X_train_vec

<40000x92691 sparse matrix of type '<class 'numpy.float64'>'
	with 3519880 stored elements in Compressed Sparse Row format>

In [ ]:
from scipy import sparse

X_train_combined = sparse.hstack([
    X_train_vec,
    X_train.loc[:, ["length"]]
    ])
X_test_combined = sparse.hstack([
    X_test_vec, 
    X_test.loc[:, ["length"]]
    ])

In [ ]:
lc_stapled = LogisticRegression()
lc_stapled.fit(X_train_combined, y_train)
lc_stapled_pred= lc_stapled.predict(X_test_combined)
lc_stapled_acc= accuracy_score(lc_stapled_pred, y_test)
print("stapling length gave us accuracy of", lc_stapled_acc)

stapling length gave us accuracy of 0.8932


In [ ]:
X_train["length_normed"] = (X_train["length"] - X_train["length"].mean()) / X_train["length"].std()
X_test["length_normed"] = (X_test["length"] - X_test["length"].mean()) / X_test["length"].std()

X_train_combined = sparse.hstack([
    X_train_vec,
    X_train.loc[:, ["length_normed"]]
    ])
X_test_combined = sparse.hstack([
    X_test_vec, 
    X_test.loc[:, ["length_normed"]]
    ])

In [ ]:
lc_stapled_norm = LogisticRegression()
lc_stapled_norm.fit(X_train_combined, y_train)
lc_stapled_norm_pred= lc_stapled.predict(X_test_combined)
lc_stapled_norm_acc= accuracy_score(lc_stapled_norm_pred, y_test)
print("norm stapling length gave us accuracy of", lc_stapled_norm_acc)

norm stapling length gave us accuracy of 0.8948


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
max_length = 100

In [ ]:
text_data = X["review"]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_data)

In [ ]:
tokenized_data = tokenizer.texts_to_sequences(text_data)
padded_data =  tf.keras.preprocessing.sequence.pad_sequences(
    tokenized_data,
    maxlen = max_length,
    padding = "post"
)

In [ ]:
padded_data

array([[   33,  1636,     8, ...,   124,  4102,   485],
       [13739,   790,  9718, ...,  1976,    68,   220],
       [    9,   189,     8, ...,    62,    15,   349],
       ...,
       [   25,     3,  1155, ..., 22839,     2,  6049],
       [    5,    67,   134, ...,    66,   738,    41],
       [    1,   115,   698, ...,   793,    10,    16]], dtype=int32)

In [ ]:
labels = np.array(y)

In [ ]:
model =  tf.keras.Sequential([
     tf.keras.layers.Embedding(
                                                input_dim = len(tokenizer.word_index)+1, 
                                                output_dim = 64,
                                                 input_length = max_length
                                                 ),
      tf.keras.layers.Conv1D(
                                            64,
                                            4,
                                            activation='relu'),
       tf.keras.layers.MaxPooling1D(pool_size = 4),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=False)),
        tf.keras.layers.Dense(1, activation = "sigmoid")                     
])


model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [ ]:
model.fit(padded_data[:100], labels[:100], epochs=10, batch_size=2)

Epoch 1/10
50/50 [==============================] - 9s 180ms/step - loss: 0.6919 - accuracy: 0.5700
Epoch 2/10
50/50 [==============================] - 8s 164ms/step - loss: 0.6406 - accuracy: 0.5800
Epoch 3/10
50/50 [==============================] - 7s 149ms/step - loss: 0.3454 - accuracy: 0.8900
Epoch 4/10
50/50 [==============================] - 8s 161ms/step - loss: 0.0585 - accuracy: 0.9900
Epoch 5/10
50/50 [==============================] - 8s 161ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 9s 167ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 9s 173ms/step - loss: 5.6311e-04 - accuracy: 1.0000
Epoch 8/10
50/50 [==============================] - 10s 208ms/step - loss: 3.4572e-04 - accuracy: 1.0000
Epoch 9/10
50/50 [==============================] - 8s 151ms/step - loss: 2.3078e-04 - accuracy: 1.0000
Epoch 10/10
50/50 [==============================] - 8s 159ms/step - loss: 1.6589e-04 -

In [ ]:
new_data = [
    "this movie was wonderful",
    "this movie was terrible",
    "the fukm was not so good, i didn't love it",
    "the movie was not great at all; i don't recommend it"
]

new_data_tok = tokenizer.texts_to_sequences(new_data)
new_padded = tf.keras.preprocessing.sequence.pad_sequences(
    new_data_tok,
    maxlen = max_length,
    padding = "post"
)

In [ ]:
predictions = model.predict(new_padded)

1/1 [==============================] - 1s 904ms/step


In [ ]:
list(map(lambda x: x.round(), list(predictions)))

[array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32),
 array([1.], dtype=float32)]

In [ ]:
with tf.device("/GPU:0"):
    sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(log_device_placement= True))
    tf.compat.v1.keras.backend.set_session(sess)

    model.fit(padded_data[:1000], labels[:1000], epochs=10, batch_size=2)
    tf.compat.v1.keras.backend.clear_session()

NameError: ignored

In [ ]:
new_data = [
    "this movie was wonderful",
    "this movie was terrible",
    "the fukm was not so good, i didn't love it",
    "the movie was not great at all; i don't recommend it"
]

new_data_tok = tokenizer.texts_to_sequences(new_data)
new_padded = tf.keras.preprocessing.sequence.pad_sequences(
    new_data_tok,
    maxlen = max_length,
    padding = "post"
)

predictions = model.predict(new_padded)

list(map(lambda x: x.round(), list(predictions)))

In [ ]:
import numpy as np

lc.coef_

array([[-0.04671427, -0.14119928, -0.03783384, ..., -0.01274101,
        -0.04190874, -0.04190874]])

In [ ]:
test_list = [
    "this movie was so bad! i hated it!",
    "this was the best movie i have ever seen!",
    "this movie was not bad at all!",
    "this movie was SO not great, i did not love it!"
]

test_vec = vectorizer.transform(test_list)
lc.predict(test_vec)


array([0, 1, 0, 1], dtype=uint8)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_vec, y_train)
mnb_pred = mnb.predict(X_test_vec)
mnb_acc = accuracy_score(mnb_pred, y_test)
print("mnb accuracy was", mnb_acc)

mnb accuracy was 0.8652


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [ ]:
processed_corpus = [simple_preprocess(doc) for doc in X["review"]]


In [ ]:
processed_corpus[0]

In [ ]:
model = Word2Vec(
                sentences = processed_corpus, 
                 vector_size=100, 
                 window = 10, 
                 min_count=1,
                 workers=4,
                 sg=0,
                 epochs=10)

In [ ]:
model.wv.most_similar(
    positive=["king","woman"],
    negative=["man"]
    )

[('queen', 0.6943681240081787),
 ('princess', 0.5865194201469421),
 ('duchess', 0.5566383600234985),
 ('carmen', 0.49354782700538635),
 ('cinderella', 0.4914223551750183),
 ('lestat', 0.4913289546966553),
 ('belle', 0.48836860060691833),
 ('gwtw', 0.4707203805446625),
 ('gypsy', 0.4670458734035492),
 ('godmother', 0.46405139565467834)]

In [ ]:
doc1 = processed_corpus[0]
words = [word for word in doc1 if word in model.wv]

In [ ]:
model.wv[words]

array([[-0.5547855 , -0.91391927,  0.48617813, ..., -0.2773611 ,
         1.9531353 , -0.16648403],
       [ 1.3998739 ,  2.7539814 ,  0.2082603 , ..., -0.81982845,
         0.45304117, -0.8420056 ],
       [ 2.410833  ,  1.4753253 ,  2.0203514 , ..., -0.536773  ,
         1.8952826 ,  0.38439965],
       ...,
       [-1.3722521 , -5.404353  , -2.204562  , ...,  0.2648533 ,
        -0.763515  , -0.81330943],
       [ 0.3799278 , -0.33049121, -0.7202109 , ...,  0.37195528,
         2.9466727 , -1.3297604 ],
       [-0.03638764, -0.2587707 ,  2.9594846 , ..., -2.242334  ,
         0.64830226, -1.1567047 ]], dtype=float32)

In [ ]:
import numpy as np

def doc_to_vector(doc, embdedding_model):
    words = [word for word in doc1 if word in embdedding_model.wv]
    if words: 
        return np.mean(embdedding_model.wv[words], axis=0)
    else:
        return np.zeros(embdedding_model.vector_size)

doc_embedded_vec = np.array([doc_to_vector(doc,model) for doc in processed_corpus])


In [ ]:
X = doc_embedded_vec
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
lr_emb = LogisticRegression()
lr_emb.fit(X_train, y_train)


LogisticRegression()

In [ ]:
lr_emb_pred=lr_emb.predict(X_test)
lr_emb_acc = accuracy_score(lr_emb_pred, y_test)
print("accuracy of logistic regression on embedded words was", lr_emb_acc)

accuracy of logistic regression on embedded words was 0.4961


In [ ]:
from gensim.downloader import load
pt_emb = load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def pt_doc_to_vector(doc, embdedding_model):
    words = [word for word in doc.lower().split() if word in embdedding_model]
    if words: 
        return np.mean(embdedding_model[words], axis=0)
    else:
        return np.zeros(embdedding_model.vector_size)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
X_train_pt_emb = np.array([pt_doc_to_vector(doc,pt_emb) for doc in X_train["review"]])
X_test_pt_emb = np.array([pt_doc_to_vector(doc,pt_emb) for doc in X_test["review"]])

In [ ]:
lc_pt_emb =  LogisticRegression(max_iter=5000)
lc_pt_emb.fit(X_train_pt_emb, y_train)
lc_pt_emb_pred=lc_pt_emb.predict(X_test_pt_emb)
lc_pt_emb_acc = accuracy_score(lc_pt_emb_pred, y_test)
print("with pretained embeddings accuracy was", lc_pt_emb_acc)

with pretained embeddings accuracy was 0.7847
